In [357]:
import pandas as pd
from finta import TA
from pathlib import Path
import random

df_demo = pd.read_csv(Path('raw_data/SPY_data.csv'))
df_demo_demo = '../raw_data/SPY_data.csv'


In [358]:
def create_df(df_ohlcv):
    # create a base dataframe from an imported csv file
    df = pd.read_csv(Path(f'raw_data/{df_ohlcv}'))
    df = df.drop(columns='Unnamed: 0')
    df.set_index('date', inplace=True)
    return df 

In [359]:
def choose_indicator(filepath, indicator):
    #Pick an indicator to addd to base dataframe
    indicator = indicator.upper() 

    df = create_df(filepath)
    if indicator == 'ATR':
        df['ATR'] = TA.ATR(df)
        
    elif indicator == 'BBANDS':
        df[['BBANDS Upper','Moving Average', 'BBands Lower']] = TA.BBANDS(df)

    elif indicator == 'VBM':
        df['VBM'] = TA.VBM(df)

    elif indicator == 'KAMA':
        df['KAMA'] = TA.KAMA(df)

    elif indicator == 'DYMI':
        df['DYMI'] = TA.DYMI(df)

    elif indicator == 'KC':
        df[['KC Upper', 'KC Lower']] = TA.KC(df)

    elif indicator == 'APZ':
        df[['APZ Upper', 'APZ Lower']] = TA.APZ(df)
       
    elif indicator == 'MI':
        df['MI'] = TA.MI(df)

    else:
        print("Indicator not a volatility indicator")
         

    return df

In [360]:

def custom_pair_indicators(filepath, indicator_1, indicator_2):
    # pair any two indicators into a single dataframe
    # need to create two instances of choose indicator 
    indicator_1 = indicator_1.upper()
    indicator_2 = indicator_2.upper()
    
    df_1 = choose_indicator(filepath, indicator_1)
    df_2 = choose_indicator(filepath, indicator_2)
    
    df = df_1.merge(df_2, how='inner', right_index=True, left_index=True)
    df = df.drop(columns=['open_y', 'high_y','low_y','close_y', 'volume_y','trade_count_y','vwap_y'])
    df = df.rename(columns={'open_x':'open', 'high_x':'high', 'low_x':'low', 'close_x':'close', 'volume_x':'volume', 'trade_count_x':'trade_count', 'vwap_x':'vwap'})
    return df

In [361]:
def drop_any_column(df, *drop_list):
    # drop any column you like
    for drop_me in drop_list:
        df = df.drop(columns=drop_me)
    return df


In [362]:
def mix_pair_indicators(df_1, df_2):
    # merge two different combinatoins of indicators into one frame
    df = df_1.merge(df_2, how='inner', right_index=True, left_index=True)
    df = df.drop(columns=['open_y', 'high_y','low_y','close_y', 'volume_y','trade_count_y','vwap_y'])
    df = df.rename(columns={'open_x':'open', 'high_x':'high', 'low_x':'low', 'close_x':'close', 'volume_x':'volume', 'trade_count_x':'trade_count', 'vwap_x':'vwap'})
    return df


In [363]:
def apply_rolling_window(df, period,*columns):
    # apply rolling window to any column
    for col in columns:
        df[f'{col}'] = df[f'{col}'].rolling(window=period).mean()
    return df


In [364]:
example = custom_pair_indicators(df_demo_demo, "VBM", 'MI')
example.iloc[20:30]

,open,high,low,close,volume,trade_count,vwap,VBM,MI
date,,,,,,,,,
2015-12-30 05:00:00+00:00,207.09,207.21,205.76,205.93,63317679,237053,206.644516,NaN,NaN
2015-12-31 05:00:00+00:00,205.17,205.89,203.87,203.89,114877859,377772,204.693408,NaN,NaN
2016-01-04 05:00:00+00:00,200.53,201.03,198.59,201.01,222353534,655482,200.645153,NaN,NaN
2016-01-05 05:00:00+00:00,201.39,201.90,200.05,201.35,110845729,418705,201.079710,NaN,NaN
2016-01-06 05:00:00+00:00,198.33,200.06,197.60,198.85,152112604,548381,198.958712,NaN,24.342927
2016-01-07 05:00:00+00:00,195.32,197.44,193.59,193.99,213436023,796441,195.345235,-67.158660,24.448476
2016-01-08 05:00:00+00:00,195.21,195.85,191.58,191.95,209567228,754911,193.646639,-66.775739,24.529962
2016-01-11 05:00:00+00:00,193.05,193.41,189.82,192.11,187941153,701513,191.722893,-67.230381,24.477138
2016-01-12 05:00:00+00:00,193.85,194.55,191.14,193.66,172330490,635729,192.886816,-69.124984,24.443229


In [365]:
def get_column_range(df,period,*columns):
        # get the rolling range of a column
        for col in columns:
                high = df[f'{col}'].rolling(window=period).max()
                low = df[f'{col}'].rolling(window=period).min()
                df[f'{col} {period} period rolling range'] = high -low
        return df


In [395]:
# create a function that takes indicators and drop columns as arguemt
# then generates random strings to get indicators merged on single dataframe
def generate_random_df(filepath, rolling_window=False, drop_basic_column=False, column_range=False):
    indicator_list = ['ATR', 'DYMI', 'APZ','KC','VBM','BBANDS','MI', 'KAMA']
    all_column_list = ['ATR', 'DYMI', 'APZ','KC','VBM','BBANDS','MI', 'KAMA', 'close','open','high','low']
    base_columns = ['close','open','high','low','trade_count','vwap']
    indicator_1 = random.choice(indicator_list)
    indicator_2 = random.choice(indicator_list)
    if indicator_2 != indicator_1:
        df = custom_pair_indicators(filepath, indicator_1, indicator_2)
        if rolling_window == True:
            random_window = random.choice(range(2,10))
            column_rolling = random.choice(base_columns)          
            # not accepting my columns perameter
            df = apply_rolling_window(df,random_window,column_rolling)
        if drop_basic_column == True:
            drop_me = random.choice(base_columns)
            df = drop_any_column(df, drop_me)
        if column_range == True:
            range_period = random.choice(range(2,10))
            df = get_column_range(df, range_period, indicator_1)
            
        # indicators with mulitple columns not passing though    

    return df 
        

In [409]:
generate_random_df(df_demo_demo, rolling_window=True,drop_basic_column=True)

,open,high,low,close,volume,trade_count,DYMI,ATR
date,,,,,,,,
2015-12-01 05:00:00+00:00,209.42,NaN,209.1100,210.68,97858418,337780,NaN,NaN
2015-12-02 05:00:00+00:00,210.60,NaN,208.2300,208.54,108069059,367013,NaN,NaN
2015-12-03 05:00:00+00:00,208.90,NaN,204.7511,205.58,166224154,546768,NaN,NaN
2015-12-04 05:00:00+00:00,206.10,NaN,205.9300,209.66,192878747,556731,NaN,NaN
2015-12-07 05:00:00+00:00,209.20,NaN,207.2000,208.27,102027111,374705,NaN,NaN
...,...,...,...,...,...,...,...,...
2022-03-24 04:00:00+00:00,445.82,443.060000,444.7600,450.48,61411569,526865,61.665813,8.861557
2022-03-25 04:00:00+00:00,451.09,446.327500,448.4300,452.69,77199328,661033,73.744729,8.262143
2022-03-28 04:00:00+00:00,452.04,448.856250,450.0600,455.91,68356914,620868,77.127858,7.816429


In [368]:
# create an explanation summary function of each time the function is ran
# then upload each epoch to a dataframe

# combine all indicators into one df and change the moving average rolling period
# two moving exponential moving averages
# look into the angle of a moving average